1. CSP로 이진분류
2. 머신러닝 기법 이용(로지스틱) 데이터 분류
3. 전처리된 데이터를 전결합층 fully connected 엡실러 층3개로 해서 해보기

In [1]:
!pip install numpy moabb

  Using cached moabb-1.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached edfio-0.4.5-py3-none-any.whl.metadata (4.3 kB)
  Using cached EDFlib_Python-1.0.8-py3-none-any.whl.metadata (1.3 kB)
  Using cached memory_profiler-0.61.0-py3-none-any.whl.metadata (20 kB)
  Using cached mne-1.9.0-py3-none-any.whl.metadata (20 kB)
  Using cached mne_bids-0.14-py2.py3-none-any.whl.metadata (4.8 kB)
     ---------------------------------------- 0.0/5.2 MB ? eta -:--:--
     ------------ --------------------------- 1.6/5.2 MB 9.3 MB/s eta 0:00:01
     ------------------ --------------------- 2.4/5.2 MB 6.1 MB/s eta 0:00:01
     ------------------------ --------------- 3.1/5.2 MB 5.3 MB/s eta 0:00:01
     ------------------------------ --------- 3.9/5.2 MB 4.8 MB/s eta 0:00:01
     ------------------------------------ --- 4.7/5.2 MB 4.6 MB/s eta 0:00:01
     ---------------------------------------- 5.2/5.2 MB 4.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependenc

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.114 requires urllib3>=2.2.2, but you have urllib3 1.26.20 which is incompatible.
dask-expr 1.1.13 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.


In [2]:
from moabb.datasets import BNCI2014_008
from moabb.paradigms import P300
import pandas as pd

In [3]:
dataset = BNCI2014_008()

In [4]:
paradigm = P300()

In [5]:
X, labels, meta = paradigm.get_data(dataset=dataset, subjects=[1], return_epochs=True)

C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\moabb\datasets\download.py:55: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_BNCI_PATH"
  set_config(key, get_config("MNE_DATA"))
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\urllib3\connectionpool.py:1064: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lampx.tugraz.at'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
100%|#####################################| 21.5M/21.5M [00:00<00:00, 21.6GB/s]
SHA256 hash of downloaded file: cdf9bca4f48c61ee9c9ba998382a4e84392918c9dbc5a43f45ee0052614e25bd
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


Adding metadata with 3 columns
Adding metadata with 3 columns
4200 matching events found
No baseline correction applied


C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\moabb\datasets\preprocessing.py:279: UserWarning: warnEpochs <Epochs | 4200 events (all good), 0 – 1 s (baseline off), ~65.9 MiB, data loaded,
 'Target': 700
 'NonTarget': 3500>
  warn(f"warnEpochs {epochs}")
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
C:\Users\kimso\anaconda3\envs\sol3\lib\site-packages\moabb\paradigms\base.py:350: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  X = mne.concatenate_epochs(X)


In [ ]:
X.shape

In [14]:
df = X.to_data_frame()

In [33]:
from IPython.display import display

display(df.head())  # 처음 1000개 행 출력

,time,condition,epoch,Fz,Cz,Pz,Oz,P3,P4,PO7,PO8
0,0.000000,NonTarget,0,-2.919284,-5.059983,-3.159821,-3.134295,-5.555135,-3.984568,-0.798101,-2.252647
1,0.003906,NonTarget,0,-0.998181,-2.105539,0.745129,4.416750,-1.525778,1.270971,5.625820,3.897016
2,0.007812,NonTarget,0,0.887403,1.012338,4.709787,12.266051,2.835857,6.565038,11.803882,10.312262
3,0.011719,NonTarget,0,2.244849,3.555139,7.796145,18.472412,6.542550,10.711572,16.357263,15.546081
4,0.015625,NonTarget,0,2.718209,4.943282,9.337894,21.522240,8.833964,12.837491,18.398629,18.435847


In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mne.decoding import CSP

In [27]:
# EEG 채널 리스트
eeg_channels = ['Fz', 'Cz', 'Pz', 'Oz', 'P3', 'P4', 'PO7', 'PO8']

# 데이터에서 필요한 채널과 라벨만 선택
df_filtered = df[eeg_channels + ['condition']]

# 라벨링 (Target = 1, NonTarget = 0)
df_filtered['label'] = df_filtered['condition'].apply(lambda x: 1 if x == 'Target' else 0)
df_filtered = df_filtered.drop(columns=['condition'])  # 원본 condition 컬럼 삭제

# 데이터 분할 (80% 학습 / 20% 테스트)
X = df_filtered[eeg_channels].values  # EEG 신호 데이터
y = df_filtered['label'].values  # 타겟 여부 (이진 분류)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 데이터 정규화 (표준화)
scaler = StandardScaler()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

C:\Users\kimso\AppData\Local\Temp\ipykernel_17152\1397099842.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['label'] = df_filtered['condition'].apply(lambda x: 1 if x == 'Target' else 0)


In [28]:
csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

In [29]:
X_train_csp = csp.fit_transform(X_train, y_train)
X_test_csp = csp.transform(X_test)

Computing rank from data with rank=None
    Using tolerance 46 (2.2e-16 eps * 8 dim * 2.6e+16  max singular value)
    Estimated rank (data): 8
    data: rank 8 computed from 8 data channels with 0 projectors
Reducing data rank from 8 -> 8
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.


In [34]:
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_csp, y_train)

# 예측 수행 (Train & Test)
y_pred_train_lda = lda.predict(X_train_csp)  # Train 데이터 예측
y_pred_test_lda = lda.predict(X_test_csp)    # Test 데이터 예측

# 정확도 계산
train_accuracy = accuracy_score(y_train, y_pred_train_lda)
test_accuracy = accuracy_score(y_test, y_pred_test_lda)

# 출력
print(f"🔹 LDA Train 정확도: {train_accuracy:.4f}")
print(f"🔹 LDA Test 정확도: {test_accuracy:.4f}")

🔹 LDA Train 정확도: 0.8333
🔹 LDA Test 정확도: 0.8335


In [ ]:
# SVM 모델 생성 (선형 커널 사용)
svm = SVC(kernel='linear', C=0.2)

# 모델 학습
svm.fit(X_train_csp, y_train)

# 예측 수행 (Train & Test)
y_pred_train_svm = svm.predict(X_train_csp)  # Train 데이터 예측
y_pred_test_svm = svm.predict(X_test_csp)    # Test 데이터 예측

In [ ]:
# 정확도 계산
train_accuracy = accuracy_score(y_train, y_pred_train_svm)
test_accuracy = accuracy_score(y_test, y_pred_test_svm)

# 출력
print(f"🔹 SVM Train 정확도: {train_accuracy:.4f}")
print(f"🔹 SVM Test 정확도: {test_accuracy:.4f}")

# 분류 보고서 출력
print("\n🔹 SVM 모델 평가 (Test Set)")
print(classification_report(y_test, y_pred_test_svm))